# Batch run for zonal stats
Tips from: https://gorelick.medium.com/fast-er-downloads-a2abd512aa26

In [89]:
import os
import multiprocessing
import numpy as np
from retry import retry
import geopandas as gpd
import ee
import geemap

In [2]:
## Register with ee using high-valume (and high-latency) endpoint
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

## Functions

In [96]:
def getRequests():
    """Generates a list of work items to be downloaded.
    Funnily enough, equivalent  to np.arange(-179.75, 179.75, 0.5)
    """    
    ## Load BAWLD
    bawld = ee.FeatureCollection('projects/ee-ekyzivat/assets/Shapes/BAWLD/bawld_grid')

    # For testing: Filter BAWLD to reduce size of operation
    bawldF = bawld.filter("Lat > 59").filter("Lat < 60") #.filter("Long == -126.25")

    ## Aggregate by Longitude
    return np.unique(bawldF.aggregate_array('Long').getInfo()) # change to bawld not bawldF for real run

In [97]:
@retry(tries=10, delay=1, backoff=2) # (tries=10, delay=1, backoff=2)
def getResult(index, long):
    """Handle the HTTP requests to download one result. index is python index and long is longitude, used for aggregation."""
    
    ## I/O
    out_dir = '/mnt/g/Ch4/GSW_zonal_stats/v0/tiles'
    out_pth = os.path.join(out_dir, f'bawld_zStats_Oc_Long{long:06.5}.csv')

    ## CRS (ist there a smarter way to do this?)
    crs = '''PROJCRS["Lambert_Azimuthal_Equal_Area",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ID["EPSG",6326]],PRIMEM["Greenwich",0,ANGLEUNIT["Degree",0.0174532925199433]]],CONVERSION["unnamed",METHOD["Lambert Azimuthal Equal Area",ID["EPSG",9820]],PARAMETER["Latitude of natural origin",90,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",south,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",south,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]'''

    ## Load BAWLD
    bawld = ee.FeatureCollection('projects/ee-ekyzivat/assets/Shapes/BAWLD/bawld_grid')

    ## Load GSW
    gsw = ee.Image('JRC/GSW1_1/GlobalSurfaceWater')
    occurrence = gsw.select('occurrence')

    ## Filter based on longitude bin, specified as variable 'long'
    bawldF = bawld.filter(ee.Filter.eq("Long", long))

    # statistics_type can be either 'SUM' or 'PERCENTAGE'
    # denominator can be used to convert square meters to other areal units, such as square kilometers
    geemap.zonal_statistics_by_group(
        occurrence,
        bawldF,
        out_pth,
        statistics_type='SUM',
        denominator=1000000,
        decimal_places=3,
        crs=crs,
        tile_scale=1 # default is 1, but it exceeds GEE capacity
    )

    print("Done with Longitude: ", long)

## Apply functions

In [98]:
import numpy as np
items = getRequests()
# len(np.unique(items))
len(items)
# items

462

In [99]:
## Run function
items = getRequests()
print(f'Sending request in {len(items)} chunks...')
pool = multiprocessing.Pool(25)
pool.starmap(getResult, enumerate(items))
pool.close()
pool.join()

Sending request in 462 chunks...
Computing ... 
Computing ... 
Computing ... Computing ... 

Computing ... Computing ... 

Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Generating URL ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Data downloaded to /mnt/g/Ch4/GSW_zonal_stats/v0/tiles/bawld_zStats_Oc_Long-114.25.csv
Done with Longitude:  -114.25
Data downloaded to /mnt/g/Ch4/GSW_zonal_stats/v0/tiles/bawld_zStats_Oc_Long-150.75.csv
Done with Longitude:  -150.75
Computing ... 
Data downloaded to /mnt/g/Ch4/GSW_zonal_stats/v0/tiles/bawld_zStats_Oc_Long-104.25.csv
Done with Longitude:  -104.25
D

In [ ]:
!watch -n 5 'ls /mnt/g/Ch4/GSW_zonal_stats/v0/tiles/'